In [1]:
from automl import AutoML

In [2]:
import openml

task = openml.tasks.get_task(31)

# estimation_parameters — словарь, но значения там строки
params = task.estimation_parameters
n_folds = int(params.get("number_folds", 1))
n_repeats = int(params.get("number_repeats", 1))

print("folds:", n_folds, "repeats:", n_repeats)

splits = []

for repeat in range(n_repeats):
    for fold in range(n_folds):
        train_idx, test_idx = task.get_train_test_split_indices(
            repeat=repeat,
            fold=fold
        )
        splits.append({
            "repeat": repeat,
            "fold": fold,
            "train_idx": train_idx,
            "test_idx": test_idx,
        })

# пример использования
dataset = task.get_dataset()
X, y, categorical, feature_names = dataset.get_data(target=dataset.default_target_attribute)

folds: 10 repeats: 1


In [3]:
from sklearn.metrics import roc_auc_score


automl = AutoML(
    task='classification',
    use_preprocessing_pipeline=False,
    feature_selector_type=None,
    use_val_test_pipeline=False,
    auto_models_init_kwargs = {
        "metric": "roc_auc",
        "time_series": False,
        "models_list": ["linear", "forests", "boostings"],
        "blend": True,
        "stack": True,
        "n_splits": 10,
    },
    n_jobs=3, 
    random_state=0,
    )
automl = automl.fit(
    X, y, 
    auto_model_fit_kwargs = {
        "tuning_timeout": 10
    }
)

[2025-11-14 18:54:21,511] - [   MODEL    ] - 1 out of 8. type
[2025-11-14 18:54:21,513] - [   START    ] - Working with type
[2025-11-14 18:54:21,522] - [   START    ] - Tuning type
[2025-11-14 18:54:31,863] - [   OPTUNA   ] - Trial 0. New best score 0.6021904761904763 with parameters {'max_iter': 8277, 'fit_intercept': True, 'C': 1.8590843630169627, 'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.9636627605010293}
[2025-11-14 18:54:31,864] - [   OPTUNA   ] - 1 trials completed
[2025-11-14 18:54:31,864] - [BEST PARAMS ] - {'random_state': 0, 'n_jobs': 3, 'verbose': 0, 'max_iter': 8277, 'fit_intercept': True, 'C': 1.8590843630169627, 'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.9636627605010293, 'time_series': False, 'model_type': 'classification', 'n_splits': 10, 'name': 'type', 'eval_metric': 'custom_metric'}
[2025-11-14 18:54:31,867] - [    END     ] - Tuning type
[2025-11-14 18:54:31,868] - [   START    ] - Fitting type
[2025-11-14 18:54:41,406] - [    END     ]

In [6]:
score = automl.auto_model.best_score

In [9]:
test_predictions = automl.predict(X)[:, 1]

In [10]:
score, test_predictions

(0.7908,
 array([0.67427265, 1.21795073, 0.44453031, 1.23496647, 1.24291544,
        0.68944788, 0.55854798, 1.08904895, 0.43046146, 1.24818164,
        1.22399769, 1.41016595, 0.94105918, 1.17934822, 1.12964185,
        1.24909721, 0.4388148 , 1.1473761 , 1.18801642, 0.4905242 ,
        0.47493858, 0.74496351, 0.79498399, 0.74119371, 0.46987823,
        0.80363   , 0.51689551, 0.66293833, 0.76959062, 1.27002304,
        0.88119288, 1.13716251, 1.0970617 , 0.50794435, 0.87069062,
        1.18734861, 0.76890967, 0.76676244, 0.69268617, 0.94803895,
        0.59396379, 0.94813698, 0.91472054, 1.03984733, 1.22786601,
        0.57703509, 0.57369148, 0.95478626, 0.54230256, 0.54640924,
        1.02052984, 0.95601354, 0.56825806, 0.4531762 , 1.20189743,
        0.54136127, 0.83277918, 0.70483692, 1.02393894, 1.35082198,
        0.83943493, 0.66164064, 1.22831159, 1.29523794, 0.60638867,
        0.54986164, 0.56758903, 1.01396517, 0.74914785, 0.6594899 ,
        0.73252531, 0.41962735, 0.86987